## Pull one symbol with one action

In [ ]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary"

querystring = {"symbol":"AAPL","region":"US"}

headers = {
	"X-RapidAPI-Key": "551595c10emsh9ab76ccfb317820p1fa121jsnc738c9455698",
	"X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

In [ ]:
import pprint
pprint.pprint(response.json()["defaultKeyStatistics"])

In [ ]:
pprint.pprint(response.json())

In [ ]:
# defaultKeyStatistics: beta (5Y Monthly), bookValue (book value per share), earningsQuarterlyGrowth (Quarterly earnings growth year-over-year), enterpriseToEbitda, enterpriseToRevenue, enterpriseValue, forwardEps (forward means using forecast earning for next 12 month), forwardPE, heldPercentInsiders, heldPercentInstitutions,
# netIncomeToCommon (income that could be given to common stockholders, ttm), pegRatio, priceToBook, trailingEps (for recent past 12 month)

# summaryProfile: sector, industry

# price: averageDailyVolume10Day, averageDailyVolume3Month, marketCap

# symbol

# financialData: currentRatio (mrq), debtToEquity (mrq), ebitda (ttm), ebitdaMargins (ttm), freeCashflow (ttm), grossMargins (ttm), grossProfits (ttm), operatingCashflow (ttm), operatingMargins (ttm), profitMargins (ttm), quickRatio (mrq), returnOnAssets (ttm), returnOnEquity (ttm), revenueGrowth (quarterly revenue growth yoy), revenuePerShare (ttm)
# totalCash (mrq), totalCashPerShare (mrq), totalDebt (mrq), totalRevenue (ttm)

# esgScores: environmentScore, esgPerformance, governanceScore, highestControversy, peerEnvironmentPerformance, peerEsgScorePerformance, peerGovernancePerformance, peerHighestControversyPerformance, peerSocialPerformance, socialScore, totalEsg

# summaryDetail: fiftyDayAverage, fiftyTwoWeekHigh, fiftyTwoWeekLow, twoHundredDayAverage, dividendRate, dividendYield, fiveYearAvgDividendYield, payoutRatio

**comment requirement: do not have "," in parenthesis.**

save this in a txt file: e.g. _stock-v2-get-summary-keys.txt_

## subset using keys

In [115]:
full = response.json()
from collections import defaultdict

read it into _content_ variable

In [117]:
file_path = "stock-v2-get-summary-keys.txt"
try:
    with open(file_path, 'r') as file:
        content = file.read()
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("An error occurred while reading the file.")

print(len(content.split('\n\n')))
items = content.split('\n\n')
items

7


['defaultKeyStatistics: beta (5Y Monthly), bookValue (book value per share), earningsQuarterlyGrowth (Quarterly earnings growth year-over-year), enterpriseToEbitda, enterpriseToRevenue, enterpriseValue, forwardEps (forward means using forecast earning for next 12 month), forwardPE, heldPercentInsiders, heldPercentInstitutions,\nnetIncomeToCommon (income that could be given to common stockholders ttm), pegRatio, priceToBook, trailingEps (for recent past 12 month)',
 'summaryProfile: sector, industry',
 'price: averageDailyVolume10Day, averageDailyVolume3Month, marketCap',
 'symbol',
 'financialData: currentRatio (mrq), debtToEquity (mrq), ebitda (ttm), ebitdaMargins (ttm), freeCashflow (ttm), grossMargins (ttm), grossProfits (ttm), operatingCashflow (ttm), operatingMargins (ttm), profitMargins (ttm), quickRatio (mrq), returnOnAssets (ttm), returnOnEquity (ttm), revenueGrowth (quarterly revenue growth yoy), revenuePerShare (ttm)\ntotalCash (mrq), totalCashPerShare (mrq), totalDebt (mrq),

In [160]:
# comment block for debug

# items = [i for i in content.split('\n') if i]
# items = content.split('\n\n')
# items

In [161]:
# comment block for debug

# item = items[0]
# item.split(":")

In [162]:
# # comment block for debug

# item2 = item.split(":")[1].split(",")
# item2
# print(item2)
# for skey in item2:
#     print(skey.strip())
#     print("extract from full | key: {}, val {}".format(skey.strip().split(' (', 1)[0],full['defaultKeyStatistics'][skey.strip().split(' (', 1)[0]]))
#     print("==========")

take subset based on keys into _subset_ variable

In [129]:
subset = defaultdict(dict)
for item in items:
    key_val = item.split(":")
    if len(key_val) == 1:
        # extract whole part for this item, such as symbol
        key = key_val[0]
        subset[key] = full[key]
    elif len(key_val) == 2:
        key, val = key_val
        # for e.g. key = defaultKeyStatistics, val = ' beta (5Y Monthly), bookValue (book value per share), earningsQuarterlyGrowth...'
        for sub_key in val.split(","):
            # for e.g. sub_key = ' beta (5Y Monthly)' or ' bookValue (book value per share)' etc.
            # first trim the white space using .strip(), then remove the parenthesis when extracting from full dict using .split()
            subset[key][sub_key.strip().replace(" ","_").replace("(","").replace(")","")] = full[key][sub_key.strip().split(' (', 1)[0]]
    else:
        raise ValueError("only support nested one layer dictionary")
    
    print("Done: {}".format(key_val[0]))


Done: defaultKeyStatistics
Done: summaryProfile
Done: price
Done: symbol
Done: financialData
Done: esgScores
Done: summaryDetail


In [130]:
# "beta (5Y Monthly)".replace(" ","_").replace("(","").replace(")","")

In [ ]:
pprint.pprint(subset)

## convert to df

In [134]:
import pandas as pd

In [ ]:
data = {'defaultKeyStatistics': {'beta_5Y_Monthly': {'fmt': '1.29',
                                                          'raw': 1.289436},
                'bookValue_book_value_per_share': {'fmt': '3.95',
                                                    'raw': 3.953},
                'earningsQuarterlyGrowth_Quarterly_earnings_growth_year-over-year': {'fmt': '-3.40%',
                                                                                    'raw': -0.034},
                'enterpriseToEbitda': {'fmt': '23.81',
                                        'raw': 23.807},
                'enterpriseToRevenue': {'fmt': '7.65',
                                        'raw': 7.653},
                'enterpriseValue': {'fmt': '2.95T',
                                    'longFmt': '2,947,037,790,208',
                                    'raw': 2947037790208},
                'forwardEps_forward_means_using_forecast_earning_for_next_12_month': {'fmt': '6.56',
                                                                                    'raw': 6.56},
                'forwardPE': {'fmt': '28.36',
                            'raw': 28.355183},
                'heldPercentInsiders': {'fmt': '0.07%',
                                        'raw': 0.00071000005},
                'heldPercentInstitutions': {'fmt': '61.23%',
                                            'raw': 0.61232},
                'netIncomeToCommon_income_that_could_be_given_to_common_stockholders_ttm': {'fmt': '94.32B',
                                                                                            'longFmt': '94,321,000,448',
                                                                                            'raw': 94321000448},
                'pegRatio': {'fmt': '3.87', 'raw': 3.87},
                'priceToBook': {'fmt': '47.06',
                                'raw': 47.055397},
                'trailingEps_for_recent_past_12_month': {'fmt': '5.90',
                                                        'raw': 5.9}},
'esgScores': {'environmentScore': {'fmt': '0.6', 'raw': 0.65},
    'esgPerformance': 'UNDER_PERF',
    'governanceScore': {'fmt': '9.2', 'raw': 9.18},
    'highestControversy': 3,}
}

data = subset
def process_leaf_numeric(value):
    # process second layer info: e.g. beta_5Y_Monthly, bookValue_book_value_per_share, ... etc.
    if not isinstance(value, dict):
        return value
    if "raw" in value:
        return value["raw"]
    if "avg" in value:
        return value['avg']

dict_concat = {}
for firstKey, dic in data.items():
    # process first layer: e.g. defaultKeyStatistics, summaryProfile, symbol, ... etc.
    print(firstKey)
    if not isinstance(dic, dict):
        dict_concat[firstKey] = dic
        continue
    flat_data = {
        key: process_leaf_numeric(value)
        for key, value in dic.items()
    }
    dict_concat.update(flat_data)
# not set up layer properly


pprint.pprint(dict_concat)

In [159]:
# Convert the flattened dictionary into a DataFrame
df = pd.DataFrame([dict_concat])

df.set_index('symbol', inplace=True)

new_col_order = ['sector', 'industry'] + [col for col in df.columns if col not in ['sector', 'industry']]
df = df[new_col_order]


# Display the DataFrame
df

,sector,industry,beta_5Y_Monthly,bookValue_book_value_per_share,earningsQuarterlyGrowth_Quarterly_earnings_growth_year-over-year,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,forwardEps_forward_means_using_forecast_earning_for_next_12_month,forwardPE,...,socialScore,totalEsg,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,twoHundredDayAverage,dividendRate,dividendYield,fiveYearAvgDividendYield,payoutRatio
symbol,,,,,,,,,,,,,,,,,,,,,
AAPL,Technology,Consumer Electronics,1.289436,3.953,-0.034,23.807,7.653,2947037790208,6.56,28.355183,...,6.86,16.68,171.5526,186.515,124.17,152.9297,0.96,0.0052,0.9,0.1559
